In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import r2_score, accuracy_score, mean_absolute_error, f1_score, ConfusionMatrixDisplay, RocCurveDisplay, classification_report

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

# Dataset

In [2]:
dataset = pd.read_csv("master_lag.csv")

In [3]:
print("Number of rows : {}".format(dataset.shape[0]))
print("Number of columns : {}".format(dataset.shape[1]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 39588
Number of columns : 43

Display of dataset: 


,Unnamed: 0,Date,Code INSEE région,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Bioénergies (MW),...,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,rolling_mean_7,rolling_mean_15,lag_364
0,4368,2013-12-31,11,476296.0,13005.0,0.0,395.0,71.0,48.0,6514.0,...,471877.0,446277.0,477600.0,510076.0,500238.0,508977.0,524368.0,446431.142857,467695.0,399392.0
1,4380,2014-01-01,11,424366.0,12694.0,0.0,655.0,41.0,48.0,7465.0,...,450994.0,471877.0,446277.0,477600.0,510076.0,500238.0,508977.0,450683.571429,461028.2,492157.0
2,4392,2014-01-02,11,455574.0,12185.0,0.0,598.0,155.0,99.0,7397.0,...,394599.0,450994.0,471877.0,446277.0,477600.0,510076.0,500238.0,450867.571429,457468.0,487111.0
3,4404,2014-01-03,11,450735.0,12003.0,0.0,818.0,124.0,123.0,7571.0,...,454286.0,394599.0,450994.0,471877.0,446277.0,477600.0,510076.0,449652.000000,454167.8,470053.0
4,4416,2014-01-04,11,425674.0,12041.0,0.0,475.0,131.0,96.0,7528.0,...,459244.0,454286.0,394599.0,450994.0,471877.0,446277.0,477600.0,450649.571429,448541.0,433732.0



Basics statistics: 


,Unnamed: 0,Date,Code INSEE région,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Bioénergies (MW),...,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,rolling_mean_7,rolling_mean_15,lag_364
count,39588.000000,39588,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000,...,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000,39588.000000
unique,NaN,3299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2013-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,24161.769172,NaN,49.916667,213116.425621,18166.027938,169452.053754,13209.988001,4956.073747,27517.843248,4156.696208,...,213186.691573,213188.703067,213197.311837,213217.169395,213243.811711,213268.933717,213289.202258,213160.924093,213193.204946,215493.682783
std,11428.671462,NaN,25.640326,100063.782946,24672.138121,181441.587109,20418.044353,6544.595255,45522.146419,2157.246976,...,100068.692039,100070.913919,100076.765409,100089.075530,100102.824304,100116.085833,100129.590967,98711.288713,98293.546000,101521.418779
min,4368.000000,NaN,11.000000,43907.500000,-1407.000000,-4231.000000,0.000000,0.000000,0.000000,393.000000,...,59592.000000,59592.000000,59592.000000,59592.000000,59592.000000,59592.000000,59592.000000,68326.571429,71764.266667,59592.000000
25%,14264.750000,NaN,27.750000,131341.500000,1248.000000,0.000000,2131.750000,862.000000,288.000000,2354.000000,...,131410.250000,131410.250000,131419.750000,131430.000000,131441.000000,131457.500000,131464.500000,129722.571429,130026.400000,132866.000000
50%,24161.500000,NaN,48.000000,195696.500000,8155.500000,121527.500000,6378.000000,2193.000000,3409.000000,3880.000000,...,195742.000000,195742.000000,195742.000000,195742.000000,195747.500000,195750.000000,195753.000000,196095.285714,196102.466667,197133.000000
75%,34058.250000,NaN,75.250000,274129.750000,23814.750000,314284.500000,15642.750000,6490.250000,40647.000000,5665.000000,...,274194.500000,274207.500000,274217.750000,274232.750000,274251.250000,274307.250000,274327.250000,276647.107143,277385.366667,277236.500000



Percentage of missing values: 


Unnamed: 0              0.000000
Date                    0.000000
Code INSEE région       0.000000
Consommation (MW)       0.000000
Thermique (MW)          0.000000
Nucléaire (MW)          0.000000
Eolien (MW)             0.000000
Solaire (MW)            0.000000
Hydraulique (MW)        0.000000
Bioénergies (MW)        0.000000
Ech. physiques (MW)     0.000000
Stockage batterie       0.000000
year                    0.000000
month                   0.000000
brent_price            30.554714
TIME_PERIOD            11.397393
prix_kwh_elec          11.397393
temp_max                0.272810
temp_min                0.272810
hours_of_sun            0.303122
precipitation           0.303122
windspeed               0.272810
prix_gaz                0.363747
day                     0.000000
day_of_week             0.000000
lag_1                   0.000000
lag_2                   0.000000
lag_3                   0.000000
lag_4                   0.000000
lag_5                   0.000000
lag_6     

In [4]:
dataset.columns

Index(['Unnamed: 0', 'Date', 'Code INSEE région', 'Consommation (MW)',
       'Thermique (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)',
       'Hydraulique (MW)', 'Bioénergies (MW)', 'Ech. physiques (MW)',
       'Stockage batterie', 'year', 'month', 'brent_price', 'TIME_PERIOD',
       'prix_kwh_elec', 'temp_max', 'temp_min', 'hours_of_sun',
       'precipitation', 'windspeed', 'prix_gaz', 'day', 'day_of_week', 'lag_1',
       'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9',
       'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'lag_15',
       'rolling_mean_7', 'rolling_mean_15', 'lag_364'],
      dtype='object')

# Train test split

In [5]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")

split_date = '2021-01-01'
train = dataset.loc[dataset['Date'] <= split_date].copy()
test = dataset.loc[dataset['Date'] > split_date].copy()

# Y_train = dataset.loc[dataset['Date'] <= split_date].copy()
# Y_test = dataset.loc[dataset['Date'] > split_date].copy()

Dividing into train and test sets...


In [6]:
# Separate target variable Y from features X
print("Separating labels from features...")
features_list = [
                'Code INSEE région'
                #'lag_0', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6'  
                #,'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'lag_364'
                #,'rolling_mean_7', 'rolling_mean_15'
                ,'temp_max', 'temp_min', 'hours_of_sun', 'precipitation', 'windspeed' 
                , 'prix_kwh_elec', 'prix_gaz', 'brent_price'
                , 'day', 'year', 'month', 'day_of_week'
                ]
target_variable = ["Consommation (MW)"]

X_train = train.loc[:,features_list]
X_test = test.loc[:,features_list]

y_train = train.loc[:,target_variable]
y_test = test.loc[:,target_variable]

Separating labels from features...


# Preprocessing

In [7]:
# Soit à la main : 
numeric_features = [
                #'lag_0', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6'  
                #'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'lag_364'
                #, 'rolling_mean_7', 'rolling_mean_15'
                 'temp_max', 'temp_min', 'hours_of_sun', 'precipitation', 'windspeed' 
                 ,'prix_kwh_elec', 'prix_gaz', 'brent_price'
                 ,'year', 'day', 'month'
                    ]
categorical_features = ['Code INSEE région', 'day_of_week']

In [8]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=1)),
    ('scaler', StandardScaler()) 
])

In [9]:
# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('imputer', KNNImputer(n_neighbors=1)),
    ('encoder', OneHotEncoder(drop='first')),
    ])

In [10]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [11]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()




# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) # Don't fit again !!
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
   Code INSEE région  temp_max  temp_min  hours_of_sun  precipitation  \
0                 11    8.4875    5.4250       2.21000         2.5750   
1                 11   10.1625    4.6250       1.42500         2.7250   
2                 11   10.1750    7.8500       3.92000         1.5875   
3                 11   12.3750    7.8625       3.53250         1.9500   
4                 11   10.8125    6.5125       2.71375         1.1875   

   windspeed  prix_kwh_elec   prix_gaz  brent_price  day  year  month  \
0    21.7250         0.1524  33.412419       109.95   31  2013     12   
1    33.9375         0.1585  29.812258          NaN    1  2014      1   
2    32.8250         0.1585  29.812258       107.94    2  2014      1   
3    32.3625         0.1585  29.812258       106.57    3  2014      1   
4    22.1250         0.1585  29.812258          NaN    4  2014      1   

   day_of_week  
0            1  
1            2  
2            3  
3           

# Model

In [12]:
sgdr = SGDRegressor(random_state=0, penalty = 'l1', max_iter = 5000)
sgdr.fit(X_train, y_train.values.ravel())
sgd_train_score = sgdr.score(X_train, y_train.values.ravel())
sgd_test_score = sgdr.score(X_test, y_test.values.ravel())
print("Train score: {}".format(sgd_train_score))
print("Test score: {}".format(sgd_test_score))

Train score: 0.929345812707287
Test score: -0.43106113959459447


In [13]:
rfr = RandomForestRegressor(random_state=0, n_jobs=-1, criterion = 'squared_error')
rfr.fit(X_train, y_train.values.ravel())
rf_train_score = rfr.score(X_train, y_train.values.ravel())
rf_test_score = rfr.score(X_test, y_test.values.ravel())
print("Train score: {}".format(rf_train_score))
print("Test score: {}".format(rf_test_score))

Train score: 0.9982671149714171
Test score: 0.9634319562510669


In [14]:
regressor = Lasso(alpha=86)
regressor.fit(X_train, y_train)
reg_train_score = regressor.score(X_train, y_train.values.ravel())
reg_test_score = regressor.score(X_test, y_test.values.ravel())
print("Train score: {}".format(reg_train_score))
print("Test score: {}".format(reg_test_score))

Train score: 0.9281833305763698
Test score: -0.21754960462752537


In [15]:
voting = VotingRegressor(estimators=[("linear", regressor), ("forest", rfr), ("sgd", sgdr)])
voting.fit(X_train, y_train)
print("R2 on training set : ", voting.score(X_train, y_train.values.ravel()))
print("R2 on test set : ", voting.score(X_test, y_test.values.ravel()))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_voting.py:600: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 on training set :  0.966390416762069
R2 on test set :  0.3641842404941158
